## 导入库和数据

In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
# oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\off_train_sameUser_id_test.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')
samplt=pd.read_csv(r'D:\Data\TCForNewComer\sample_submission.csv')

In [2]:
off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,4,1433,8735,30:5,10,20160214,null
1,4,1469,2902,0.95,10,20160607,null
2,35,3381,1807,300:30,0,20160130,null
3,35,3381,9776,10:5,0,20160129,null
4,35,3381,11951,200:20,0,20160129,null


In [3]:
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [4]:
on_train=on_train.sort_values(by=['User_id'])
on_train.index=np.arange(0,len(on_train),1)
on_train.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,4,25104,2,100145044,100:10,20160331,null
1,4,45612,1,null,null,null,20160308
2,36,46701,0,null,null,null,20160120
3,64,11200,0,null,null,null,20160526
4,64,29214,0,null,null,null,20160606


## 选取间隔日期大于15天，标记为负样本

In [5]:
index=off_train[(off_train['Date']!='null')&(off_train['Coupon_id']=='null')].index
ordinary=off_train.loc[index]
ordinary['Date']=0
ordinary['Coupon_id']=0
ordinary['Discount_rate']=0
ordinary['Date_received']=0
ordinary['Coupon_id']=ordinary['Coupon_id'].astype(np.int64)
ordinary['Date_received']=ordinary['Date_received'].astype(np.int64)
ordinary['Discount_rate']=ordinary['Discount_rate'].astype(np.object)
ordinary['Date']=ordinary['Date'].astype(np.int64)

In [6]:
p_train=off_train[(off_train['Date']!='null')&(off_train['Coupon_id']!='null')]#正样本
n_train=off_train[(off_train['Date']=='null')&(off_train['Coupon_id']!='null')]#负样本

p_train.index=np.arange(0,len(p_train),1)
n_train.index=np.arange(0,len(n_train),1)
#转换为时间格式
p_train['Date_received']=pd.to_datetime(p_train['Date_received'])
p_train['Date']=pd.to_datetime(p_train['Date'])
n_train['Date_received']=pd.to_datetime(n_train['Date_received'])
test['Date_received']=pd.to_datetime(test['Date_received'])

p_train['date']=(p_train['Date']-p_train['Date_received']).astype('timedelta64[D]')
p_train.loc[p_train[p_train['date']<=15].index,'date']=1
p_train.loc[p_train[p_train['date']>15].index,'date']=0
p_train['Date']=p_train['date']
p_train.drop(['date'],axis=1,inplace=True)
p_train['Date']=p_train['Date'].astype(np.int64)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [ ]:
str(p_train.iloc[0]['Date_received']).split('-')[1][1]+str(p_train.iloc[0]['Date_received']).split('-')[2][:2]

In [ ]:
p_train['Date_received']=p_train['Date_received'].apply(lambda x: str(x).split('-')[1][1]+str(x).split('-')[2][:2])#转换日期
n_train['Date_received']=n_train['Date_received'].apply(lambda x: str(x).split('-')[1][1]+str(x).split('-')[2][:2])
test['Date_received']=test['Date_received'].apply(lambda x: str(x).split('-')[1][1]+str(x).split('-')[2][:2])
#正样本为1，负样本为0
p_train.loc[:,'Date']=1
n_train.loc[:,'Date']=0

train=pd.concat([p_train,n_train],axis=0)
train=train.sort_values(by=['User_id'])

In [ ]:
import seaborn as sns
%matplotlib inline

sns.countplot(x='Date_received',hue='Date',data=train)

In [7]:
p_train['Date_received']=p_train['Date_received'].apply(lambda x: x.weekday()+1)#转换为星期几
n_train['Date_received']=n_train['Date_received'].apply(lambda x: x.weekday()+1)
test['Date_received']=test['Date_received'].apply(lambda x: x.weekday()+1)
#正样本为1，负样本为0
n_train.loc[:,'Date']=0

train=pd.concat([p_train,n_train,ordinary],axis=0)
train=train.sort_values(by=['User_id'])
train.index=np.arange(0,len(train),1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [ ]:
train['Date_received']=train['Date_received'].apply(lambda x: 1 if x<=5 else 0)
test['Date_received']=test['Date_received'].apply(lambda x: 1 if x<=5 else 0)

## 转化满减为小数折扣

In [8]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return round((float(x.split(':')[0])-float(x.split(':')[1]))/float(x.split(':')[0]),2)

In [9]:
def Discount_rate(x):#转化函数
    if str(x).startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])

In [10]:
train=train.replace('null',np.nan)
test=test.replace('null',np.nan)

In [11]:
#将满减转化为小数
train['Discount_rate']=train['Discount_rate'].apply(Discount_rate)

In [12]:
train=train.fillna(method='ffill')

In [13]:
train['Distance']=train['Distance'].astype(np.int64)
train['Coupon_id']=train['Coupon_id'].astype(np.int64)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        int64
Discount_rate    float64
Distance         int64
Date_received    int64
Date             int64
dtypes: float64(1), int64(6)
memory usage: 107.1 MB


## 不同类型数据的影响

In [ ]:
train['Merchant_id']=train['Merchant_id'].astype(np.object)
train['User_id']=train['User_id'].astype(np.object)
train['Date_received']=train['Date_received'].astype('category')
test['Merchant_id']=test['Merchant_id'].astype(np.object)
test['Coupon_id']=test['Coupon_id'].astype(np.object)
test['Distance']=test['Distance'].astype(np.object)
test['Date_received']=test['Date_received'].astype('category')

In [ ]:
test['User_id']=test['User_id'].astype(np.object)

### 填充测试集的空缺值

In [15]:
test=test.fillna(method='ffill')

In [16]:
test['Discount_rate']=test['Discount_rate'].apply(Discount_rate)

In [17]:
test['Merchant_id']=test['Merchant_id'].astype(np.int64)
test['Coupon_id']=test['Coupon_id'].astype(np.int64)
test['Distance']=test['Distance'].astype(np.int64)

In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113640 entries, 0 to 113639
Data columns (total 6 columns):
User_id          113640 non-null int64
Merchant_id      113640 non-null int64
Coupon_id        113640 non-null int64
Discount_rate    113640 non-null float64
Distance         113640 non-null int64
Date_received    113640 non-null int64
dtypes: float64(1), int64(5)
memory usage: 6.1 MB


## 距离统计分布

In [ ]:
import seaborn as sns 
%matplotlib inline

sns.countplot(x='Distance',hue='Date',data=train)

In [ ]:
import seaborn as sns
%matplotlib inline

sns.countplot(x='Date_received',hue='Date',data=train)

In [ ]:
import seaborn as sns 
%matplotlib inline
plt.figure(figsize=(10,6))
sns.countplot(x='Discount_rate',hue='Date',data=train)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train.head()

In [ ]:
a=set(train['Coupon_id'].values)
c=set(test['Coupon_id'].values)
len(c)

In [ ]:
corr=train.corr()
corr

In [ ]:
combine=[train,test]
for data in combine:
    data.drop(['Date_received'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
a=set(train['Merchant_id'].values)
c=set(test['Merchant_id'].values)
len(c)

In [ ]:
len(a&c)

# 训练模型

In [19]:
K=10
from sklearn.metrics import roc_auc_score,auc
from time import sleep
import winsound
from sklearn.model_selection import StratifiedKFold
# kf = KFold(n_splits = K,random_state = 1,shuffle = True)
kf=StratifiedKFold(n_splits = K,random_state = 90,shuffle = True)
X=train.drop(['Date'],axis=1)
y=train['Date']

## 归一化

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x_predict=test
# 1. min max scaler
min_max_scaler = MinMaxScaler()
x_predict_min_max_scaled = min_max_scaler.fit_transform(x_predict)

# 2. standard scaler
std_scaler = StandardScaler()
x_predict_std_scaled = std_scaler.fit_transform(x_predict)

In [ ]:
# 1. min max scaler
min_max_scaler = MinMaxScaler()
# x_train_min_max_scaled = min_max_scaler.fit_transform(x_train)
# x_test_min_max_scaled = min_max_scaler.transform(x_test)
X_test_min_max_scaled=min_max_scaler.fit_transform(X)#整个数据集
# 2. standard scaler
std_scaler = StandardScaler()
# x_train_std_scaled = std_scaler.fit_transform(x_train)
# x_test_std_scaled = std_scaler.transform(x_test)
X_test_std_scaled=std_scaler.fit_transform(X)

In [ ]:
import xgboost as xgb
#全部  0.5552 learning_rate=0.09
model=xgb.XGBClassifier(n_estimators=200,
                        max_depth=2,#6
                        objective="binary:logistic",
                        learning_rate=0.09, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.4,
                        scale_pos_weight=1.6,
                        gamma=9,
                        seed=100,
                        reg_alpha=8,
                        reg_lambda=1.3)

In [20]:
def Avg_auc(pred):
    aucc=0
    i=0
    df=train.copy()
    df['y_valid_pred']=pred
    for j in list(set(df['Coupon_id'].values)):
        df_1=df[df['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
            continue
        aucc=aucc+roc_auc_score(df_1['Date'],df_1['y_valid_pred'])
        i+=1
    aucc=aucc/i
    df.drop(['y_valid_pred'],axis=1,inplace=True)
    return aucc

In [21]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
y_test_pred=model.predict_proba(test)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

Avg_auc for full training set: 0.51141124345


In [ ]:
#行最大最小归一化
XX=X.copy()
for col in ['Merchant_id','Coupon_id']:
    max=XX[col].max()
    min=XX[col].min()
    XX[col]=XX[col].apply(lambda x: ((x-min)/(max-min)))

In [22]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
y_test_pred=model.predict_proba(test)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

Avg_auc for full training set: 0.893584631873


In [31]:
from sklearn.linear_model.logistic import LogisticRegression
model=LogisticRegression(C=1)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
y_test_pred=model.predict_proba(test)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

Avg_auc for full training set: 0.494281539351


In [ ]:
from sklearn.ensemble import AdaBoostClassifier#星期0.626
model=AdaBoostClassifier(n_estimators=400,learning_rate=1,random_state=100)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

In [ ]:
y_test_pred=0
y_valid_pred=0*y
fit_model=[]
# XMIN=pd.DataFrame(X_test_std_scaled,columns=['Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received'])
for i,(train_index,test_index) in enumerate(kf.split(X,y)):
    y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
    x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
    print('\nFlod:',i,end=':')
    fitted_model=model.fit(x_train,y_train)
    pred= fitted_model.predict_proba(x_valid)[:,1]
    y_valid_pred.iloc[test_index]=pred
    print(pd.DataFrame(pred).head(1))
    
    y_test_pred+=fitted_model.predict_proba(test)[:,1]
    fit_model.append(fitted_model)
    winsound.Beep(600,1000)
    sleep(1)
y_test_pred/=K
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [25]:
tt=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')

In [26]:
sub=pd.DataFrame()
sub['User_id']=tt['User_id']
sub['Coupon_id']=tt['Coupon_id']
sub['Date_received']=tt['Date_received']

In [27]:
sub['Probability']=y_test_pred

In [28]:
sub.to_csv(r'D:\Data\TCForNewComer\pred_result.csv',float_format='%.1f',index=False)

In [ ]:
y_train['a']=np.random.normal(len(y_train))

In [ ]:
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 1, 0, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
y_true.astype(np.int64)

In [ ]:
train.head()